In [1]:
from copy import deepcopy

import numpy as np
from numpy.random import randint, multinomial

from utilities import create_laplace_profile, kmer_probabilitiy, score_motifs, select_random_motifs

In [28]:
def sub_motifs(motifs, i):
    motifs_i = deepcopy(motifs[:i])
    motifs_i.extend(deepcopy(motifs[i+1:]))
    return motifs_i

def profile_randomly_generated_kmer(dna_string, profile):
    k = profile.shape[1]
    kmer_probs = []
    for i in range(0, len(dna_string)-k+1):
        kmer = dna_string[i:i+k]
        kmer_probs.append(kmer_probabilitiy(kmer, profile))
    kmer_probs /= sum(kmer_probs)
    outcomes = multinomial(1, kmer_probs)
    selected_index = np.nonzero(outcomes)[0][0]
    return dna_string[selected_index:selected_index+k]

In [17]:
k = 8
t = 5
N = 100
dna_strings = [
    "CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA",
    "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
    "TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
    "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
    "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"
]

In [29]:
def gibbs_sampler_main(dna_strings, k, t, N):
    motifs = select_random_motifs(dna_strings, k)
    best_motifs = deepcopy(motifs)
    best_score = score_motifs(motifs)
    for j in range(N):
        i = randint(t)
        motifs_i = sub_motifs(motifs, i)
        profile = create_laplace_profile(motifs_i)
        motifs[i] = profile_randomly_generated_kmer(dna_strings[i], profile)
        score = score_motifs(motifs)
        if score < best_score:
            best_motifs = deepcopy(motifs)
            best_score = score
    return best_motifs

def gibbs_sampler(dna_strings, k, t, N, epochs=20):
    motifs = gibbs_sampler_main(dna_strings, k, t, N)
    best_motifs = deepcopy(motifs)
    best_score = score_motifs(motifs)
    for j in range(epochs):
        motifs = gibbs_sampler_main(dna_strings, k, t, N)
        score = score_motifs(motifs)
        if score < best_score:
            best_motifs = deepcopy(motifs)
            best_score = score
    return best_motifs

In [31]:
result = gibbs_sampler(dna_strings, k, t, N)

In [32]:
expected_result = {
    "TCTCGGGG",
    "CCAAGGTG",
    "TACAGGCG",
    "TTCAGGTG",
    "TCCACGTG"
}
assert set(result) == expected_result

In [35]:
with open("datasets/dataset_163_4.txt","r") as fin:
    dna = []
    for i, line in enumerate(fin):
        if i==0:
            k, _, N = line.strip().split(" ")
            k = int(k)
            N = int(N)
        else:
            dna.append(line.strip())
    t = len(dna)

In [37]:
result = gibbs_sampler(dna, k, t, N)

In [38]:
for i in result:
    print(i)

TCCAATTCTCTCCCG
CCCCGAGCATGATCG
CCCATGTCATGATCG
CCCAATAGGTGATCG
CTATATGCATGATCG
CCCAATGCATTTACG
CCTCTTGCATGATCG
CCCAAAAAATGATCG
GTTAATGCATGATCG
CCCATATCATGATCG
CCCAATCTTTGATCG
ATCAATGCATGATCC
CCCAATGCACACTCG
CCCAAGCTATGATCG
GCCAATGCATGATGC
CCCAATGCATGACGA
CCCAATGCTGCATCG
CCCAATGTCGGATCG
CCCGTCGCATGATCG
CCCAATGCATGTGTG
